In [1]:
import time
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
from pandas.api.types import is_numeric_dtype
from functools import reduce
from pandas.api.types import is_numeric_dtype
from itertools import groupby
from collections import OrderedDict

In [2]:
df = pd.read_csv('../data/Echo/echo-5-lost.csv')

In [3]:
df

Age Pericardial Fractional     EPSS    LVDD Wall-score Wall-index  \
0        71           0       0.26        9     4.6         14          1   
1        72           0       0.38        6     4.1         14        1.7   
2        55           0       0.26        4    3.42         14          1   
3        60           0      0.253   12.062       ?         16       1.45   
4        57           0       0.16       22    5.75          ?       2.25   
5        68           0       0.26        5    4.31         12          1   
6        62           0       0.23       31    5.43       22.5      1.875   
7        60           0       0.33        8    5.25         14          1   
8        46           0       0.34        0    5.09         16       1.14   
9        54           0       0.14       13    4.49       15.5       1.19   
10       77           0       0.13       16    4.23         18        1.8   
11       62           1       0.45        9     3.6         16       1.14   
12       73           0          ?        6       4         14          1   
13        ?           0       0.15       10    3.73         14          1   
14       62           0       0.12       23     5.8      11.67       2.33   
15       55           1       0.25   12.063    4.29         14          1   
16       69           1       0.26       11    4.65         18       1.64   
17        ?           1       0.07       20     5.2         24          2   
18       66           0       0.09       17   5.819          8      1.333   
19       66           ?       0.22       15     5.4         27       2.25   
20       69           0       0.15       12    5.39       19.5      1.625   
21       85           1       0.18       19    5.46          ?       1.38   
22       73           0       0.23   12.733    6.06        7.5        1.5   
23       71           0       0.17        0    4.65          8          1   
24       55           1       0.21      4.2    4.16         14       1.56   
25       63           0   0.241021       10   4.649         14       1.17   
26       61           0       0.61     13.1    4.07         13      1.625   
27       63           1   0.241021  11.0272    5.31          5          1   
28       65           0       0.06     23.6  5.1145       21.5       2.15   
29        ?           0       0.51  11.0272    3.88         15       1.67   
30       54           0       0.35      9.3    3.63         11      1.222   
31       70           1       0.27      4.7    4.49         22          2   
32       79           0          ?     17.5       ?         13        1.3   
33       56           0       0.33  11.0272    3.59         14          1   
34       81           0       0.12  11.0272   4.649          9       1.25   
35       59           0       0.03     21.3    6.29         17       1.31   
36        ?           1   0.171136  15.8968  5.1145         23        2.3   
37       56           ?          ?       14       5    18.7787    1.79283   
38       61           1       0.27  15.8968       ?          9        1.5   
39       58           0        0.3      9.4    3.49         14          1   
40       60           0       0.01     24.6    5.65         39          ?   
41       66           0       0.29     15.6    6.15         14          1   
42       63           0       0.15        ?    4.57         13       1.08   
43        ?           0       0.13     18.6    4.37      12.33       1.37   
44        ?           0        0.1      9.8     5.3         23        2.3   
45       79           0       0.17     11.9    5.15       10.5       1.05   
46       72           0      0.187       12    5.02         13       1.18   
47       51           0       0.16     13.2    5.26         11          ?   
48       70           1       0.25      9.7    5.57        5.5        1.1   
49       65           0       0.36      8.8    5.78         12          1   
50       78           0       0.06     16.1    5.62      13.67      1.367   
51  

In [4]:
df_headers = list(df)
concept = df_headers[-1]
concept

'Alive'

In [5]:
#all unique concepts
concept_list = df[concept].unique()
concept_list

array([0, 1])

In [6]:
#calculating cases by concepts and making sets
U = [] #universal list containing all cases
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        U.append(index+1)
        if row[concept] == item:
            temp_list.append(index)
    goal_list.append(temp_list)
    temp_list = []

In [7]:
print(goal_list,)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 15, 18, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58, 61, 62, 64, 67, 68, 70, 71], [10, 14, 16, 17, 19, 20, 21, 22, 23, 28, 36, 37, 38, 40, 51, 56, 59, 60, 63, 65, 66, 69, 72, 73]]


In [8]:
attributes = list(df)
attributes

['Age',
 'Pericardial',
 'Fractional',
 'EPSS',
 'LVDD',
 'Wall-score',
 'Wall-index',
 'Alive']

In [9]:
case_list = []

In [10]:
def discretize(numeric_col):
    
    #Sorting the values of numeric column
    sort_col = df[numeric_col].sort_values()
   
    df['sort_col'] = sort_col.values
    point_list = df['sort_col'].unique()
    point_list = list(point_list)
    if '?' in point_list:
        point_list.remove('?')
    
    
    #Finding average between each two points
    avg_list = []
    for i in range(len(point_list)-1):
        avg = (float(point_list[i]) + float(point_list[i+1]))/2
        avg_list.append(round(float(avg),2))
        
   
    #Performing the discretization and adding the cases
    for i in avg_list:
        case = str(numeric_col) + "," + str(round(float(point_list[0]),1)) + ".." + str(i)
        case2 = str(numeric_col) + "," + str(i) + ".." + str(round(float(point_list[len(point_list)-1]),1))
        case_list.append(case)
        case_list.append(case2)

In [11]:
#Discretization considering upto 2 decimal point
for item in attributes[:-1]:
        discretize(item)

In [12]:
case_list = list(OrderedDict.fromkeys(case_list))

In [13]:
case_list

['Age,46.0..48.5',
 'Age,48.5..86.0',
 'Age,46.0..52.5',
 'Age,52.5..86.0',
 'Age,46.0..54.5',
 'Age,54.5..86.0',
 'Age,46.0..55.5',
 'Age,55.5..86.0',
 'Age,46.0..56.5',
 'Age,56.5..86.0',
 'Age,46.0..57.5',
 'Age,57.5..86.0',
 'Age,46.0..58.5',
 'Age,58.5..86.0',
 'Age,46.0..59.5',
 'Age,59.5..86.0',
 'Age,46.0..60.5',
 'Age,60.5..86.0',
 'Age,46.0..61.5',
 'Age,61.5..86.0',
 'Age,46.0..62.5',
 'Age,62.5..86.0',
 'Age,46.0..63.5',
 'Age,63.5..86.0',
 'Age,46.0..64.5',
 'Age,64.5..86.0',
 'Age,46.0..65.5',
 'Age,65.5..86.0',
 'Age,46.0..66.73',
 'Age,66.73..86.0',
 'Age,46.0..67.73',
 'Age,67.73..86.0',
 'Age,46.0..68.5',
 'Age,68.5..86.0',
 'Age,46.0..69.5',
 'Age,69.5..86.0',
 'Age,46.0..70.5',
 'Age,70.5..86.0',
 'Age,46.0..71.5',
 'Age,71.5..86.0',
 'Age,46.0..72.5',
 'Age,72.5..86.0',
 'Age,46.0..73.5',
 'Age,73.5..86.0',
 'Age,46.0..75.5',
 'Age,75.5..86.0',
 'Age,46.0..77.5',
 'Age,77.5..86.0',
 'Age,46.0..78.5',
 'Age,78.5..86.0',
 'Age,46.0..80.0',
 'Age,80.0..86.0',
 'Age,46

In [14]:
df_headers = list(df)
concept = df_headers[-2]
concept_list = df[concept].unique()

In [15]:
concept_list

array([0, 1])

In [16]:
#calculating cases by concepts and making sets
temp_list = []
goal_list = []
for item in concept_list:
    for index, row in df.iterrows():
        if row[concept] == item:
            temp_list.append(index+1)
    goal_list.append(temp_list)
    temp_list = []

In [17]:
print(goal_list,)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72], [11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61, 64, 66, 67, 70, 73, 74]]


In [18]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
  
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] is not '?':
                if float(row[a]) >= float(start) and float(row[a]) <= float(end):
                    temp_list.append(index+1)
        
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if float(case) == float(b):
                        temp_list.append(index+1)
                        
            if float(row[a]) == float(b):
                temp_list.append(index+1)
       
        att_val_list.append(temp_list)
        temp_list = []

In [19]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       Age,46.0..48.5      [9]
1       Age,48.5..86.0      [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
2       Age,46.0..52.5      [9, 48]
3       Age,52.5..86.0      [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
4       Age,46.0..54.5      [9, 10, 31, 48, 56, 59]
5       Age,54.5..86.0      [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 46, 47, 49, 50, 51, 52, 53, 54, 55, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74]
6       Age,46.0..55.5      [3, 9, 10, 16, 25, 31

In [20]:
#Creating data for case and att-value list
data = {'Cases': case_list, 'att_val': att_val_list}

In [21]:
df2 = pd.DataFrame(data)

In [22]:
#Cases and corresponding att-value pairs
df2

Cases                                            att_val
0            Age,46.0..48.5                                                [9]
1            Age,48.5..86.0  [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1...
2            Age,46.0..52.5                                            [9, 48]
3            Age,52.5..86.0  [1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 1...
4            Age,46.0..54.5                            [9, 10, 31, 48, 56, 59]
5            Age,54.5..86.0  [1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 15, 16, 1...
6            Age,46.0..55.5             [3, 9, 10, 16, 25, 31, 48, 56, 59, 62]
7            Age,55.5..86.0  [1, 2, 4, 5, 6, 7, 8, 11, 12, 13, 15, 17, 19, ...
8            Age,46.0..56.5  [3, 9, 10, 16, 25, 31, 34, 38, 48, 53, 56, 59,...
9            Age,56.5..86.0  [1, 2, 4, 5, 6, 7, 8, 11, 12, 13, 15, 17, 19, ...
10           Age,46.0..57.5  [3, 5, 9, 10, 16, 25, 31, 34, 38, 48, 53, 56, ...
11           Age,57.5..86.0  [1, 2, 4, 6, 7, 8, 11, 12, 13, 15, 17, 19, 20,...
12           Age,46.0..58.5  [3, 5, 9, 10, 16, 25, 31, 34, 38, 40, 48, 53, ...
13           Age,58.5..86.0  [1, 2, 4, 6, 7, 8, 11, 12, 13, 15, 17, 19, 20,...
14           Age,46.0..59.5  [3, 5, 9, 10, 16, 25, 31, 34, 36, 38, 40, 48, ...
15           Age,59.5..86.0  [1, 2, 4, 6, 7, 8, 11, 12, 13, 15, 17, 19, 20,...
16           Age,46.0..60.5  [3, 4, 5, 8, 9, 10, 16, 25, 31, 34, 36, 38, 40...
17           Age,60.5..86.0  [1, 2, 6, 7, 11, 12, 13, 15, 17, 19, 20, 21, 2...
18           Age,46.0..61.5  [3, 4, 5, 8, 9, 10, 16, 25, 27, 31, 34, 36, 38...
19           Age,61.5..86.0  [1, 2, 6, 7, 11, 12, 13, 15, 17, 19, 20, 21, 2...
20           Age,46.0..62.5  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 25, 27, 31,...
21           Age,62.5..86.0  [1, 2, 6, 11, 13, 17, 19, 20, 21, 22, 23, 24, ...
22           Age,46.0..63.5  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 25, 26, 27,...
23           Age,63.5..86.0  [1, 2, 6, 11, 13, 17, 19, 20, 21, 22, 23, 24, ...
24           Age,46.0..64.5  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 25, 26, 27,...
25           Age,64.5..86.0  [1, 2, 6, 11, 13, 17, 19, 20, 21, 22, 23, 24, ...
26           Age,46.0..65.5  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 25, 26, 27,...
27           Age,65.5..86.0  [1, 2, 6, 11, 13, 17, 19, 20, 21, 22, 23, 24, ...
28          Age,46.0..66.73  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 19, 20, 25,...
29          Age,66.73..86.0  [1, 2, 6, 11, 13, 17, 21, 22, 23, 24, 32, 33, ...
30          Age,46.0..67.73  [3, 4, 5, 7, 8, 9, 10, 12, 15, 16, 19, 20, 25,...
31          Age,67.73..86.0  [1, 2, 6, 11, 13, 17, 21, 22, 23, 24, 32, 33, ...
32           Age,46.0..68.5  [3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 19, 20, ...
33           Age,68.5..86.0  [1, 2, 11, 13, 17, 21, 22, 23, 24, 32, 33, 35,...
34           Age,46.0..69.5  [3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 19, ...
35           Age,69.5..86.0  [1, 2, 11, 13, 22, 23, 24, 32, 33, 35, 46, 47,...
36           Age,46.0..70.5  [3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 19, ...
37           Age,70.5..86.0  [1, 2, 11, 13, 22, 23, 24, 33, 35, 46, 47, 51,...
38           Age,46.0..71.5  [1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17, 1...
39           Age,71.5..86.0  [2, 11, 13, 22, 23, 33, 35, 46, 47, 51, 52, 65...
40           Age,46.0..72.5  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 17...
41           Age,72.5..86.0       [11, 13, 22, 23, 33, 35, 46, 51, 52, 65, 73]
42           Age,46.0..73.5  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16...
43           Age,73.5..86.0               [11, 22, 33, 35, 46, 51, 52, 65, 73]
44           Age,46.0..75.5  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16...
45           Age,75.5..86.0                   [11, 22, 33, 35, 46, 51, 52, 73]
46           Age,46.0..77.5  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...
47           Age,77.5..86.0                       [22, 33, 35, 46, 51, 52, 73]
48           Age,46.0..78.5  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15...
49           Age,78.5..86.0                               [22, 33, 35, 4

### Lower/Upper Approximation !!

In [23]:
attributes = list(df)
del attributes[-1]

In [24]:
attributes

['Age',
 'Pericardial',
 'Fractional',
 'EPSS',
 'LVDD',
 'Wall-score',
 'Wall-index',
 'Alive']

In [25]:
case_list = []
#Loop through all the attributes except last 2 - Concept and sort_col
for item in attributes[:-1]:
    print(item)
    
    #check for non numeric columns
    if not is_numeric_dtype(df[item]):
        temp = df[item].unique()
        for i in temp:
            if i == '?' or i == '-':
                continue
            else:
                case = item + "," + i
                case_list.append(case)

Age
Pericardial
Fractional
EPSS
LVDD
Wall-score
Wall-index


In [26]:
case_list

['Age,71',
 'Age,72',
 'Age,55',
 'Age,60',
 'Age,57',
 'Age,68',
 'Age,62',
 'Age,46',
 'Age,54',
 'Age,77',
 'Age,73',
 'Age,69',
 'Age,66',
 'Age,85',
 'Age,63',
 'Age,61',
 'Age,65',
 'Age,70',
 'Age,79',
 'Age,56',
 'Age,81',
 'Age,59',
 'Age,58',
 'Age,51',
 'Age,78',
 'Age,86',
 'Age,67.4578',
 'Age,64',
 'Age,74',
 'Pericardial,0',
 'Pericardial,1',
 'Fractional,0.26',
 'Fractional,0.38',
 'Fractional,0.253',
 'Fractional,0.16',
 'Fractional,0.23',
 'Fractional,0.33',
 'Fractional,0.34',
 'Fractional,0.14',
 'Fractional,0.13',
 'Fractional,0.45',
 'Fractional,0.15',
 'Fractional,0.12',
 'Fractional,0.25',
 'Fractional,0.07',
 'Fractional,0.09',
 'Fractional,0.22',
 'Fractional,0.18',
 'Fractional,0.17',
 'Fractional,0.21',
 'Fractional,0.241021',
 'Fractional,0.61',
 'Fractional,0.06',
 'Fractional,0.51',
 'Fractional,0.35',
 'Fractional,0.27',
 'Fractional,0.03',
 'Fractional,0.171136',
 'Fractional,0.3',
 'Fractional,0.01',
 'Fractional,0.29',
 'Fractional,0.1',
 'Fractional,

In [27]:
temp_list = []
att_val_list = []
for item in case_list:
    a,b = item.split(",") #a = attribute and b = value
    if ".." in b:
        start,end = b.split("..")
        for index, row in df.iterrows():
            if row[a] >= int(start) and row[a] <= int(end):
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []
        
    else:
        for index, row in df.iterrows():
            if type(row[a]) == list:
                tmp_list = row[a]
                for case in tmp_list:
                    if case == b:
                        temp_list.append(index+1)
                        
            if row[a] == b:
                temp_list.append(index+1)
        print(temp_list)
        att_val_list.append(temp_list)
        temp_list = []

[1, 24]
[2, 47]
[3, 16, 25, 62]
[4, 8, 41, 54]
[5, 72]
[6]
[7, 12, 15, 74]
[9]
[10, 31, 56, 59]
[11]
[13, 23]
[17, 21]
[19, 20, 42, 68]
[22]
[26, 28, 43, 70]
[27, 39, 61, 64]
[29, 50, 66]
[32, 49, 69]
[33, 46]
[34, 38, 53]
[35]
[36, 55, 63, 71]
[40, 67]
[48]
[51, 73]
[52]
[57]
[58]
[65]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 19, 21, 23, 24, 26, 27, 29, 30, 31, 33, 34, 35, 36, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 65, 67, 68, 69, 70, 71, 72, 73, 74]
[12, 16, 17, 18, 22, 25, 28, 32, 37, 39, 49, 59, 66]
[1, 3, 6, 17, 57, 74]
[2, 69]
[4]
[5, 48]
[7, 23, 73]
[8, 34]
[9]
[10]
[11, 44]
[12]
[14, 21, 43]
[15, 35, 54]
[16, 49, 53]
[18, 59]
[19]
[20]
[22]
[24, 46, 67]
[25]
[26, 28]
[27]
[29, 51]
[30]
[31]
[32, 39]
[36]
[37, 61]
[40, 70]
[41]
[42, 55]
[45]
[47]
[50]
[52]
[56]
[58, 64, 65, 68]
[60]
[62]
[63]
[72]
[1, 12]
[2, 13]
[3]
[4]
[5]
[6]
[7]
[8]
[9, 24, 69]
[10]
[11]
[14, 26, 60]
[15]
[16]
[17, 53]
[18]
[19]
[20]
[21, 47]
[22]
[23]


In [28]:
#Print two evenly distributed list side by side
fmt = '%-8s%-20s%s'

for i, (case, att_val) in enumerate(zip(case_list, att_val_list)):
    print(fmt % (i, case, att_val))

0       Age,71              [1, 24]
1       Age,72              [2, 47]
2       Age,55              [3, 16, 25, 62]
3       Age,60              [4, 8, 41, 54]
4       Age,57              [5, 72]
5       Age,68              [6]
6       Age,62              [7, 12, 15, 74]
7       Age,46              [9]
8       Age,54              [10, 31, 56, 59]
9       Age,77              [11]
10      Age,73              [13, 23]
11      Age,69              [17, 21]
12      Age,66              [19, 20, 42, 68]
13      Age,85              [22]
14      Age,63              [26, 28, 43, 70]
15      Age,61              [27, 39, 61, 64]
16      Age,65              [29, 50, 66]
17      Age,70              [32, 49, 69]
18      Age,79              [33, 46]
19      Age,56              [34, 38, 53]
20      Age,81              [35]
21      Age,59              [36, 55, 63, 71]
22      Age,58              [40, 67]
23      Age,51              [48]
24      Age,78              [51, 73]
25      Age,86              [52]

In [29]:
U = set(U)
print(U,)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}


In [30]:
# Creating dictionary combining case_list and att_val list
block = dict(zip(case_list, att_val_list))
block

{'Age,71': [1, 24],
 'Age,72': [2, 47],
 'Age,55': [3, 16, 25, 62],
 'Age,60': [4, 8, 41, 54],
 'Age,57': [5, 72],
 'Age,68': [6],
 'Age,62': [7, 12, 15, 74],
 'Age,46': [9],
 'Age,54': [10, 31, 56, 59],
 'Age,77': [11],
 'Age,73': [13, 23],
 'Age,69': [17, 21],
 'Age,66': [19, 20, 42, 68],
 'Age,85': [22],
 'Age,63': [26, 28, 43, 70],
 'Age,61': [27, 39, 61, 64],
 'Age,65': [29, 50, 66],
 'Age,70': [32, 49, 69],
 'Age,79': [33, 46],
 'Age,56': [34, 38, 53],
 'Age,81': [35],
 'Age,59': [36, 55, 63, 71],
 'Age,58': [40, 67],
 'Age,51': [48],
 'Age,78': [51, 73],
 'Age,86': [52],
 'Age,67.4578': [57],
 'Age,64': [58],
 'Age,74': [65],
 'Pericardial,0': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  15,
  19,
  21,
  23,
  24,
  26,
  27,
  29,
  30,
  31,
  33,
  34,
  35,
  36,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  60,
  61,
  62,
  63,
  65,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74],
 '

In [31]:
dic = {}
for index, row in df.iterrows():
    tmp_set = set()
    final_union = []
    char_list = []
    char_list_2 = []
    final_union_set = []
  
    for cols in attributes[:-1]:
        #If the value for corresponding attribute is a list then create all of the att-value pairs
        if type(df.loc[index,cols]) == list:
            print("When values are list") #########
            for item in df.loc[index,cols]:
                block_key = cols + "," + item
                char_list.append(block_key) #char_list has all att-val cases
                print(char_list) #########
                
            union_set = set()
            #Compute union of att-concept value case
            for item in char_list:
                union_set = union_set.union(set(block[item]))
            
            print("Union Set: ", union_set) #########
            final_union.append(union_set)
        
        else:
            print("When value is single") #########
            block_key = cols + "," + str(df.loc[index,cols])
            char_list_2.append(block_key) #char_list_2 has all single cases
            print(char_list_2) ##########
   
    #Compute instersection for this current row for Characteristics set
    
    print("final_union: ", final_union) ########
    if len(final_union):
        final_union_set = list(reduce(set.intersection, [set(item) for item in final_union]))
        
    
    print("Final Union Set: ", final_union_set) ##########
    
    for item in char_list_2:
        if item in block:
            print("When item is found as key in the block") ########
            if tmp_set == set():
                #Copy over the current set elements to B
                for i in range(len(block[item])):
                    tmp_set.add(block[item][i])
                    
            tmp_set = tmp_set.intersection(set(block[item]))
            print(tmp_set) ########
        
        #If item not in block
        else:
            print("When item is not found as key in the block") ########
            print(tmp_set) ########
            print(U) #########
            if tmp_set == set():
                tmp_set = U
                
            tmp_set = tmp_set.intersection(U)
            print(tmp_set) ########
    
    print("Final tmp_set: ", tmp_set) #########
    final_union_set = set(final_union_set)
    
    if final_union_set == set():
        tmp_set = tmp_set
    else:
        tmp_set = tmp_set.intersection(final_union_set)
        
    print("This is the final value: ", tmp_set) ###########
        
    key = ('K_%d' % (index+1))
    print(key) #########
    dic[key] = tmp_set
    print("\n") ##########

When value is single
['Age,71']
When value is single
['Age,71', 'Pericardial,0']
When value is single
['Age,71', 'Pericardial,0', 'Fractional,0.26']
When value is single
['Age,71', 'Pericardial,0', 'Fractional,0.26', 'EPSS,9']
When value is single
['Age,71', 'Pericardial,0', 'Fractional,0.26', 'EPSS,9', 'LVDD,4.6']
When value is single
['Age,71', 'Pericardial,0', 'Fractional,0.26', 'EPSS,9', 'LVDD,4.6', 'Wall-score,14']
When value is single
['Age,71', 'Pericardial,0', 'Fractional,0.26', 'EPSS,9', 'LVDD,4.6', 'Wall-score,14', 'Wall-index,1']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{24, 1}
When item is found as key in the block
{24, 1}
When item is found as key in the block
{1}
When item is found as key in the block
{1}
When item is found as key in the block
{1}
When item is found as key in the block
{1}
When item is found as key in the block
{1}
Final tmp_set:  {1}
This is the final value:  {1}
K_1


When value is single
['Age,72']
When value is sing

{20}
When item is found as key in the block
{20}
Final tmp_set:  {20}
This is the final value:  {20}
K_20


When value is single
['Age,69']
When value is single
['Age,69', 'Pericardial,0']
When value is single
['Age,69', 'Pericardial,0', 'Fractional,0.15']
When value is single
['Age,69', 'Pericardial,0', 'Fractional,0.15', 'EPSS,12']
When value is single
['Age,69', 'Pericardial,0', 'Fractional,0.15', 'EPSS,12', 'LVDD,5.39']
When value is single
['Age,69', 'Pericardial,0', 'Fractional,0.15', 'EPSS,12', 'LVDD,5.39', 'Wall-score,19.5']
When value is single
['Age,69', 'Pericardial,0', 'Fractional,0.15', 'EPSS,12', 'LVDD,5.39', 'Wall-score,19.5', 'Wall-index,1.625']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{17, 21}
When item is found as key in the block
{21}
When item is found as key in the block
{21}
When item is found as key in the block
{21}
When item is found as key in the block
{21}
When item is found as key in the block
{21}
When item is found as ke

['Age,70', 'Pericardial,1', 'Fractional,0.27', 'EPSS,4.7']
When value is single
['Age,70', 'Pericardial,1', 'Fractional,0.27', 'EPSS,4.7', 'LVDD,4.49']
When value is single
['Age,70', 'Pericardial,1', 'Fractional,0.27', 'EPSS,4.7', 'LVDD,4.49', 'Wall-score,22']
When value is single
['Age,70', 'Pericardial,1', 'Fractional,0.27', 'EPSS,4.7', 'LVDD,4.49', 'Wall-score,22', 'Wall-index,2']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{32, 49, 69}
When item is found as key in the block
{32, 49}
When item is found as key in the block
{32}
When item is found as key in the block
{32}
When item is found as key in the block
{32}
When item is found as key in the block
{32}
When item is found as key in the block
{32}
Final tmp_set:  {32}
This is the final value:  {32}
K_32


When value is single
['Age,79']
When value is single
['Age,79', 'Pericardial,0']
When value is single
['Age,79', 'Pericardial,0', 'Fractional,?']
When value is single
['Age,79', 'Pericardial,0', 

Final Union Set:  []
When item is found as key in the block
{33, 46}
When item is found as key in the block
{33, 46}
When item is found as key in the block
{46}
When item is found as key in the block
{46}
When item is found as key in the block
{46}
When item is found as key in the block
{46}
When item is found as key in the block
{46}
Final tmp_set:  {46}
This is the final value:  {46}
K_46


When value is single
['Age,72']
When value is single
['Age,72', 'Pericardial,0']
When value is single
['Age,72', 'Pericardial,0', 'Fractional,0.187']
When value is single
['Age,72', 'Pericardial,0', 'Fractional,0.187', 'EPSS,12']
When value is single
['Age,72', 'Pericardial,0', 'Fractional,0.187', 'EPSS,12', 'LVDD,5.02']
When value is single
['Age,72', 'Pericardial,0', 'Fractional,0.187', 'EPSS,12', 'LVDD,5.02', 'Wall-score,13']
When value is single
['Age,72', 'Pericardial,0', 'Fractional,0.187', 'EPSS,12', 'LVDD,5.02', 'Wall-score,13', 'Wall-index,1.18']
final_union:  []
Final Union Set:  []
When

['Age,65', 'Pericardial,1', 'Fractional,?']
When value is single
['Age,65', 'Pericardial,1', 'Fractional,?', 'EPSS,8.5']
When value is single
['Age,65', 'Pericardial,1', 'Fractional,?', 'EPSS,8.5', 'LVDD,5.47']
When value is single
['Age,65', 'Pericardial,1', 'Fractional,?', 'EPSS,8.5', 'LVDD,5.47', 'Wall-score,16']
When value is single
['Age,65', 'Pericardial,1', 'Fractional,?', 'EPSS,8.5', 'LVDD,5.47', 'Wall-score,16', 'Wall-index,1.45']
final_union:  []
Final Union Set:  []
When item is found as key in the block
{50, 29, 66}
When item is found as key in the block
{66}
When item is not found as key in the block
{66}
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}
{66}
When item is found as key in the block
{66}
When item is found as key in the bloc

In [32]:
dic

{'K_1': {1},
 'K_2': {2},
 'K_3': {3},
 'K_4': {4},
 'K_5': {5},
 'K_6': {6},
 'K_7': {7},
 'K_8': {8},
 'K_9': {9},
 'K_10': {10},
 'K_11': {11},
 'K_12': {12},
 'K_13': {13},
 'K_14': {14},
 'K_15': {15},
 'K_16': {16},
 'K_17': {17},
 'K_18': {18},
 'K_19': {19},
 'K_20': {20},
 'K_21': {21},
 'K_22': {22},
 'K_23': {23},
 'K_24': {24},
 'K_25': {25},
 'K_26': {26},
 'K_27': {27},
 'K_28': {28},
 'K_29': {29},
 'K_30': {30},
 'K_31': {31},
 'K_32': {32},
 'K_33': {33},
 'K_34': {34},
 'K_35': {35},
 'K_36': {36},
 'K_37': {37},
 'K_38': {38},
 'K_39': {39},
 'K_40': {40},
 'K_41': {41},
 'K_42': {42},
 'K_43': {43},
 'K_44': {44},
 'K_45': {45},
 'K_46': {46},
 'K_47': {47},
 'K_48': {48},
 'K_49': {49},
 'K_50': {50},
 'K_51': {51},
 'K_52': {52},
 'K_53': {53},
 'K_54': {54},
 'K_55': {55},
 'K_56': {56},
 'K_57': {57},
 'K_58': {58},
 'K_59': {59},
 'K_60': {60},
 'K_61': {61},
 'K_62': {62},
 'K_63': {63},
 'K_64': {64},
 'K_65': {65},
 'K_66': {66},
 'K_67': {67},
 'K_68': {68}

In [33]:
def lowerApproximation(charac_set,concept):
    #set to contain lower approximations
    lower = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
  
        if set_value.issubset(set(concept)):
            lower = lower.union(set_value)
          
    return lower

In [34]:
lower_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    lower_approximations[str(item)] = lowerApproximation(dic,item)

In [35]:
lower_goal_list=list(lower_approximations.values())
print(lower_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [36]:
def upperApproximation(charac_set,concept):
    #set to contain lower approximations
    upper = set()
    
    #Check for each element of the concept
    for item in concept:
        key = 'K_%d' % (item)
        set_value = charac_set[key]
        upper = upper.union(set_value)
          
    return upper

In [37]:
upper_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    upper_approximations[str(item)] = upperApproximation(dic,item)

In [38]:
upper_goal_list=list(upper_approximations.values())
print(upper_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [39]:
first_column = list(dic.keys())

In [40]:
second_column = list(dic.values())

In [41]:
prob_approx = pd.DataFrame(
    {'charset_name': first_column,
     'charset_value': second_column
    })

In [42]:
def probabilisticApproximation(concept):
    prob = []
    for index, row in prob_approx.iterrows():
        if len(row['charset_value']) != 0:
            probability_conditional = len(row['charset_value'].intersection(set(concept))) / len(row['charset_value'])
        else:
            probability_conditional = 0.0
            
        prob.append(round(probability_conditional,2))
    return prob

In [43]:
def findMiddleApprox(concept):
    probapprox = set()
    cond_prob = probabilisticApproximation(concept) #Need to put the goal
    prob_approx['cond_probability'] = cond_prob
    for index, row in prob_approx.iterrows():
        part1,part2 = row['charset_name'].split("_")
       
        if row['cond_probability'] >= 0.50:
            if int(part2) in concept:
#                 print(row['charset_value'])
                probapprox = probapprox.union(row['charset_value'])
    return probapprox   

In [44]:
middle_approximations = {}
for item in goal_list:
    #Key is the string converted list so as to add as dictionary key
    middle_approximations[str(item)] = findMiddleApprox(item)

In [45]:
middle_goal_list=list(middle_approximations.values())
print(middle_goal_list)

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 19, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 58, 59, 62, 63, 65, 68, 69, 71, 72}, {64, 66, 67, 70, 73, 74, 11, 15, 17, 18, 20, 21, 22, 23, 24, 29, 37, 38, 39, 41, 52, 57, 60, 61}]


In [46]:
df3=df2

In [47]:
import pylab as pl

In [48]:
def findGoalIntersect(goal):
    goalIntersect = []
    
    for index, row in df3.iterrows():
        #List containing intersection of (a,v) pairs and goal
        goalIntersect.append(set(row['att_val']).intersection(set(goal)))
          
    #Check if goal_intersect column exists
    if 'goal_intersect' in df3:
        df3['goal_intersect'] = goalIntersect
    else:
        #Insert new column with the recent iteration
        df3.insert(2, 'goal_intersect', goalIntersect)

In [49]:
def updateGoalIntersect(goal):
    for index, row in df3.iterrows():
        if row['goal_intersect'] != set():
            row['goal_intersetct'] = set(row['att_val']).intersection(set(goal))

In [50]:
def findCases(df3):
    case_to_be = []
    #Find the cases with maximum goal coverage
    m = max(df3['goal_intersect'], key=len)
    possible_cases = [i for i, j in enumerate(df3['goal_intersect'].tolist()) if j == m]
    
    #Index of the case covering max goal and having min no. of elements
    new_df = df3.iloc[possible_cases,:]
    
    m1 = min(new_df['att_val'], key=len)
    
    for index,row in new_df.iterrows():
        if row['att_val'] == m1:
            case_to_be.append(index)
   
    return case_to_be[0]

In [51]:
def combineInterval(test_condition):
    
    test_num = [] #This will contain the conditions having interval
    test_str = [] #This will contain the conditions having no interval
    
    #Loop through to seprate contions having intervals and no intervals
    for item in test_condition:
        if ".." in item:
            test_num.append(item)
        else:
            test_str.append(item)
   
    #Group the conditions having interval based on same attributes
    grouped = [list(g) for k, g in groupby(test_num, lambda s: s.partition(',')[0])]
    
    final_list = []
    
    #Actually combining the intervals
    for list1 in grouped:
        greatest = 0
        smallest = 0
        for item in list1:
            part1,part2 = item.split(",")
            start,stop = part2.split("..")
            start = float(start)
            stop = float(stop)

            if greatest == 0 and smallest == 0:
                greatest = start
                smallest = stop

            if start > greatest:
                greatest = float(start)

            if stop < smallest:
                smallest = float(stop)

        con_tmp = part1 + "," + str(greatest) + ".." + str(smallest)
        final_list.append(con_tmp)  
            
    actual_condition = final_list + test_str
    
    return actual_condition

In [52]:
def dropCondition(condition,current_goal):
    
    for item in range(0,len(condition)):
        temp_att_val = []
        temp_cond = condition.copy() #use list.copy() as equal operator simply copies over the reference
        temp_cond.remove(condition[item])
       
        #temp_cond contains the elements after removing the current element
        for i in temp_cond:
            if i is not None:
                location = df3.index[df3['Cases'] == i].tolist()
                element = df3['att_val'].loc[location[0]]
                temp_att_val.append(set(element))
          
        #temp_att_val contains the actual value set of the corresponding cases
        #Find the intersection if the set has more than one element, otherwise no need
        if len(temp_att_val) > 1:
            intersection = set.intersection(*temp_att_val)
        
            #if the set still remains a subset of the original goal after removing current element
            #then set the current element to None as we want to drop this later
            if intersection.issubset(current_goal):
                condition[item] = None
    
    condition = [x for x in condition if x is not None]
   
    return condition   

In [53]:
def stepAlgo(df3,selected_case,current_goal,B,condition,concept_curr):
    
    rule_set = []
    original_goal = current_goal.copy()
    
    
    while current_goal != set():
        #Check if the selected case is a subset of the current goal
        #List of current case
        A = df3['att_val'].loc[selected_case] 

        if B == set():
            #Copy over the current set elements to B
            for i in range(len(A)):
                B.add(A[i])

        #Elements of intersection of current and previous set
        A = set(A).intersection(B)
        B = A.copy()
        
        #Check if intersecting elements are subset of Goal
        if A.issubset(original_goal):
            #Current goal is updated after discarding the already covered goal by new rule
            if len(A) != 0:
                current_goal = set(current_goal) - df3['goal_intersect'].loc[selected_case]
            else:
                current_goal = set()
            

            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the conditions of a Rule
            condition.append(curr_case)
            
            #Check for possibility of dropping conditions
            if len(condition) > 1:
                condition = dropCondition(condition,original_goal)
            
            #Combine the interval
            if len(condition) > 1:
                condition = combineInterval(condition)
            

            #Join conditions
            cond = ""
            for item in condition:
                cond = cond + "(" + str(item) + ")" + " & "

            cond = cond[:-2] + "->"
            rule = cond + " (" + concept + "," + str(concept_curr) + ")"
            rule_set.append(rule)
            
            #Reset everythng and continue for covering rest of the goal
            condition = []
            B = set()
            findGoalIntersect(current_goal)
            selected_case = findCases(df3)
            
        #If not a subset of current goal
        else:
            #Assign empty set for the selected case for next iteration
            df3['goal_intersect'].loc[selected_case] = set()
           
            #Extract the current case
            curr_case = df3['Cases'].loc[selected_case]
            #Add the case to the condition list
            condition.append(curr_case)
            
            #Check for Range overlapping of the remaining cases
            if ".." in curr_case:
                second_part = curr_case.split(',')[1]
                start = float(second_part.split('..')[0])
                end = float(second_part.split('..')[1])
                for index, row in df3.iterrows():
                    if ".." in row['Cases']:
                        part2 = row['Cases'].split(',')[1]
                        start1 = float(part2.split('..')[0])
                        end1 = float(part2.split('..')[1])

                        #Assign blank set for cases with overlapping ranges
                        if set((pl.frange(start,end))).issubset(pl.frange(start1,end1)) == True:
                            if start == start1 and end <= end1:
                                row['goal_intersect'] = set()
                                
            updateGoalIntersect(A.intersection(current_goal))
            selected_case = findCases(df3)
                                   
    return rule_set

In [54]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(middle_goal_list)):
    findGoalIntersect(list(middle_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,middle_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [55]:
print("Time to run the algorithm (for Middle Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_5_lost_middle.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Middle Approximation):  78.832 Sec


['(LVDD,3.4..4.73) & (Wall-score,10.0..14.5) & (Wall-index,1.02..2.3) -> (Alive,0)', '(LVDD,4.3..4.34) -> (Alive,0)']
['(EPSS,0.0..29.95) & (Fractional,0.0..0.26) & (Age,46.0..78.5) & (Fractional,0.07..0.6) & (LVDD,4.26..6.7) & (Wall-score,10.0..33.5) & (Wall-index,1.35..2.3) -> (Alive,1)', '(Age,60.5..62.5) & (Fractional,0.0..0.18) -> (Alive,1)', '(EPSS,7.55..7.8) -> (Alive,1)']


In [56]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(lower_goal_list)):
    findGoalIntersect(list(lower_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,lower_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [57]:
print("Time to run the algorithm (for Lower Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_5_lost_lower.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Lower Approximation):  82.744 Sec


['(LVDD,3.4..4.73) & (Wall-score,10.0..14.5) & (Wall-index,1.02..2.3) -> (Alive,0)', '(LVDD,4.3..4.34) -> (Alive,0)']
['(EPSS,0.0..29.95) & (Fractional,0.0..0.26) & (Age,46.0..78.5) & (Fractional,0.07..0.6) & (LVDD,4.26..6.7) & (Wall-score,10.0..33.5) & (Wall-index,1.35..2.3) -> (Alive,1)', '(Age,60.5..62.5) & (Fractional,0.0..0.18) -> (Alive,1)', '(EPSS,7.55..7.8) -> (Alive,1)']


In [58]:
#concept_list and goal_list has 1:1 mapping
final_rules = []
start_time = time.time()

#Running algorithm for all the goals - Lower Approximation/concepts
for i in range(0,len(upper_goal_list)):
    findGoalIntersect(list(upper_goal_list[i]))
    condition = []
    B = set()
    selected_case = findCases(df3)
    
    rule_set = stepAlgo(df3,selected_case,upper_goal_list[i],B,condition,concept_list[i])
    final_rules.append(rule_set)
    print("End of goal")
    
elapsed_time = time.time() - start_time

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: MatplotlibDeprecationWarning: numpy.arange


End of goal
End of goal


In [59]:
print("Time to run the algorithm (for Upper Approximation): ", round(elapsed_time, 3), "Sec")
print("\n")
print(*final_rules, sep='\n')

with open('../data/Echo/Rules/Echo_5_lost_upper.txt', 'w') as f:
    for item in final_rules:
        f.write("%s\n" % item)

Time to run the algorithm (for Upper Approximation):  82.385 Sec


['(LVDD,3.4..4.73) & (Wall-score,10.0..14.5) & (Wall-index,1.02..2.3) -> (Alive,0)', '(LVDD,4.3..4.34) -> (Alive,0)']
['(EPSS,0.0..29.95) & (Fractional,0.0..0.26) & (Age,46.0..78.5) & (Fractional,0.07..0.6) & (LVDD,4.26..6.7) & (Wall-score,10.0..33.5) & (Wall-index,1.35..2.3) -> (Alive,1)', '(Age,60.5..62.5) & (Fractional,0.0..0.18) -> (Alive,1)', '(EPSS,7.55..7.8) -> (Alive,1)']
